# Wokflow  **TEST** con Full Bayesiana 

## Inicializacion

In [1]:
# limpio la memoria
Sys.time()
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

[1] "2025-11-07 11:09:40 EST"

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,649584,34.7,1439644,76.9,1134347,60.6
Vcells,1206410,9.3,8388608,64.0,1924957,14.7


In [ ]:
PARAM <- list()
PARAM$experimento <- "apo-001"
PARAM$semilla_primigenia <- 100343

In [3]:
setwd("/content/buckets/b1/exp")
experimento_folder <- PARAM$experimento
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

## Preprocesamiento

### Traigo df ya modificado en workflow A

In [ ]:
library(duckdb)
setwd("~/Project_Wednesday")

con <- dbConnect(duckdb::duckdb(), dbdir=":memory:")
dataset <- dbGetQuery(con, "SELECT * FROM read_parquet('df.parquet')")
dbDisconnect(con)

head(dataset)

[1] "2025-11-07 11:09:40 EST"

Loading required package: data.table



,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,758301,40.5,1439644,76.9,1439644,76.9
Vcells,722128781,5509.5,1017372130,7762.0,845986918,6454.4


[1] "2025-11-07 11:09:59 EST"

## Modelado

### Target Engineering

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

library(data.table)
setDT(dataset)

dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)]

### Training Strategy

In [ ]:
# Meses con los cuales voy a validar dentro de la optimización
PARAM$trainingstrategy$testing <- c(202103)

# Meses para optimizar
PARAM$trainingstrategy$training <- c(
  202101, 202102
)

PARAM$trainingstrategy$undersampling <- 0.1


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [21]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

#### Registros  cambio las proporciones de POS/NEG

In [22]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar := runif(nrow(dataset))]
dataset[, training := 0L]

dataset[  foto_mes %in%  PARAM$trainingstrategy$training &
  (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]


### Optimizacion de Hipeparámetros

Se optimizan los hiperparámetros maximizando la ganancia.

In [23]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

In [24]:
dtrain <- lgb.Dataset(
  data= data.matrix(dataset[training == 1L, campos_buenos, with = FALSE]),
  label= dataset[training == 1L, clase01],
  free_raw_data= TRUE
)

cat("filas", nrow(dtrain), "columnas", ncol(dtrain), "\n")
Sys.time()

filas 215803 columnas 1241 


[1] "2025-11-07 11:17:25 EST"

In [25]:
# defino los datos de testing
dataset_test <- dataset[foto_mes %in% PARAM$trainingstrategy$testing]

# precalculo el campo de la ganancia
dataset_test[, gan := -20000.0 ]
dataset_test[ clase_ternaria=="BAJA+2", gan := 780000]

# precalculo la test_matrix
test_matrix <- data.matrix(dataset_test[, campos_buenos, with= FALSE])

cat("filas", nrow(dataset_test), "columnas", ncol(dataset_test), "\n")
Sys.time()

filas 163418 columnas 1246 


[1] "2025-11-07 11:17:28 EST"

In [26]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers

Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames




In [27]:
# Especificacion de la cantidad de iteraciones de la Bayesian Optimization
# 50 es razonable
PARAM$hipeparametertuning$BO_iteraciones <- 30 # un 50 seria mas razonable

In [28]:
# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "custom",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  extra_trees = FALSE,

  max_depth = -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split = 0.0, # min_gain_to_split >= 0.0
  min_sum_hessian_in_leaf = 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1 = 0.0, # lambda_l1 >= 0.0
  lambda_l2 = 0.0, # lambda_l2 >= 0.0

  bagging_fraction = 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction = 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction = 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance = FALSE, #
  scale_pos_weight = 1.0, # scale_pos_weight > 0.0

  drop_rate = 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop = 50, # <=0 means no limit
  skip_drop = 0.5, # 0.0 <= skip_drop <= 1.0

  max_bin= 31
)

In [29]:
# Notar que se recorren algunos hiperparametros en forma logaritmica
#   y que con forbidden se tiene en cuenta el juego que hay entre min_data_in_leaf y num_leaves

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeNumericParam("num_iterations", lower= 0.0, upper= 11.1, trafo= function(x) as.integer( round(2^x)) ),
  makeNumericParam("learning_rate", lower= -8.0, upper= -1.0, trafo= function(x) 2^x ),
  makeNumericParam("feature_fraction", lower= 0.05, upper= 1.0 ),
  makeNumericParam("min_data_in_leaf", lower= 0.0, upper= log2(nrow(dtrain)/2), trafo= function(x) as.integer(round(2^x)) ),
  makeNumericParam("num_leaves", lower= 1.0, upper= 10.0, trafo= function(x) as.integer(round(2^x)) ),
  forbidden= quote( (2^min_data_in_leaf)*(2^num_leaves) > nrow(dtrain) )
)

El parámetro **ksemillerio** indica se se hace semillerio DENTRO de la bayesiana
* 1 **no** se hace Ensemble Semillerio, apenas se corre un solo LightGBM
* mayor a 1, se hace un  k-Ensemble Semillerio

El parámetro  **repe** indica si dentro de la bayesiana se toman varias medidas y luego se promedian
<br> Esto se hace ya sea que se llama a un solo LightGBM o se haceun Ensemble Semillerio de LightGBMs
<br> Tener en cuenta que repe multiplica linealmente el tiempo de corrida de la Bayesian Optimization

In [ ]:
PARAM$hipeparametertuning$ksemillerio <- 1L
PARAM$hipeparametertuning$repe <- 1L

In [31]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [32]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$BO$semillas <- sample(primos)[seq( PARAM$hipeparametertuning$ksemillerio * PARAM$hipeparametertuning$repe )]

cat( PARAM$BO$semillas)

974411

In [33]:
if(!require("rlist")) install.packages("rlist")
require("rlist")

Loading required package: rlist



In [34]:
# logueo al archivo BO_log.txt
loguear  <- function( reg, arch=NA, verbose=TRUE )
{
  t0 <- Sys.time()
  archivo <- arch
  if( is.na(arch) ) archivo <- paste0( folder, substitute( reg), ext )


  if( !file.exists( archivo ) )
  {
    # Escribo los titulos
    linea  <- paste0( "fecha\t", 
                      paste( list.names(reg), collapse="\t" ), "\n" )

    cat( linea, file=archivo )
  }

  # escribo el registro
  linea  <- paste0( format(t0, "%Y%m%d.%H%M%S"),  "\t",     # la fecha y hora
                    gsub( ", ", "\t", toString( reg ) ),  "\n" )

  cat( linea, file=archivo, append=TRUE )  # grabo al archivo

  if( verbose )  cat( linea )   # imprimo por pantalla
}

In [35]:
# esto esta en una funcion para que el garbage collector lo libere
# entrena, aplica el modelo a testing, y devuelve el vector de probabilidades

OneTrainPredict <- function(param_completo) {
    
  modelo <- lgb.train(
    data= dtrain,
    param= param_completo
  )
  gmodelo <<- modelo

  # aplico el modelo a los datos nuevos
  pred <- predict(
    modelo,
    test_matrix
  )

  return( pred )
}

In [36]:
# En el argumento x llegan los parametros de la bayesiana
#  devuelve la ganancia en datos de testing

# aqui se ira guardando la mejor iteracion de la bayesiana
gmejor <- list()
gmejor$iter <- 0
gmejor$gan <- -Inf

giter <- 0
if( file.exists("BO_log.txt") ){
  tb_BO <- fread("BO_log.txt")
  giter <- nrow(tb_BO) -1 
}

EstimarGanancia_lightgbm <- function(x) {

  giter <<- giter + 1
  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  vgan_mesetas <- c()  # las ganancias, tengo repe de ellas

  # loop de las repeticionies
  for( repe in seq( PARAM$hipeparametertuning$repe ) )
  {
     desde <- (PARAM$hipeparametertuning$repe-1)*PARAM$hipeparametertuning$ksemillerio + 1
     hasta <- desde + PARAM$hipeparametertuning$ksemillerio -1
     rsemillas <- PARAM$BO$semillas[ desde:hasta ]

     # vector inicial de probabilidades
     vpred_acum <- rep( 0.0, nrow(dataset_test) )

     # loop del semillerio
     for( sem in rsemillas ) # itero semillerio
     {
        param_completo$seed <- sem  # asigno se semilla
        vpred_acum <- vpred_acum + OneTrainPredict( param_completo )
        
        gc(full= TRUE, verbose= FALSE)
     }

     # Calculo de ganancia suavizada de la meseta
     tb_prediccion <- dataset_test[, list(gan)]
     tb_prediccion[, prob := vpred_acum ]

     setorder(tb_prediccion, -prob)
     tb_prediccion[, gan_acum := cumsum(gan)]

     # la meseta es un punto, mil para la izquierda, otros mil para la derecha
     tb_prediccion[, gan_meseta :=
       frollmean(
         x= gan_acum, n= 2001, align= "center",
         na.rm= TRUE, hasNA= TRUE
      )
     ]

     vgan_mesetas <- c(vgan_mesetas, tb_prediccion[, max(gan_meseta, na.rm = TRUE)] )
  }

  gan_mesetas_prom <- mean( vgan_mesetas ) 

  if( gan_mesetas_prom > gmejor$gan ){
    gmejor$gan <<- gan_mesetas_prom
    gmejor$iter <<- giter

    # hrabo importancia de variables
    fwrite( lgb.importance(gmodelo),
      file= paste0("impo_", giter, ".txt"),
      sep= "\t"
    )
  }

  # datos qeu voy a loguear
  xx <- copy(param_completo)
  xx$iter <- giter
  xx$metrica_mejor <- gmejor$gan
  xx$metrica_sd <- sd(vgan_mesetas)
  xx$metrica <- gan_mesetas_prom

  loguear( xx, "BO_log.txt")
  set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")  # le reordeno a mlrMBO

  return( gan_mesetas_prom ) #tiempo_corrida) )
}

In [37]:
# Aqui comienza la configuracion de la Bayesian Optimization
#  es compleja la configuracion de una Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hipeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hipeparametertuning$BO_iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)

Sys.time()

[1] "2025-11-07 11:17:29 EST"

#### Corrida de la Bayesian Optimization

In [38]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Sys.time()

Warning message in mboContinue(kbayesiana):
“Tuning ended with term.iter. No need to continue. Simply returning stored result.”


[1] "2025-11-07 11:17:29 EST"

## Produccion

Las decisiones que se toman para la construccion del modelo final son:
* Obviamente los datos donde se aplica el modelo es el mes  {202108} , que no tiene clase
* Los positvos son  POS={"BAJA+1", "BAJA+2"}, esta es una meticulosa decisión.
* Se entrena en los treinta meses del intervalo [201901, 202106]
* Se realiza undersampling al 10%
* Se utilizan los hiperparámetros optimos encontrados en la Bayesian Optimization
   * Se escala min_data_in_leaf

* Por experimentos en meses anteriores, se decide cortar en los 11000 registros con mayor probabildiad de POS={"BAJA+1", "BAJA+2"}, , esta es una *enorme* decisión.

### Final Training Strategy

In [ ]:
PARAM$train_final$future <- c(202104)

PARAM$train_final$training <- c(
  202101, 202102, 202103
)

PARAM$train_final$undersampling <- 0.10

In [40]:
# se filtran los meses donde se entrena el modelo final
dataset_train_final <- dataset[foto_mes %in% PARAM$train_final$training]

#### Registros cambio las proporciones de POS/NEG

In [41]:
# Undersampling, van todos los "BAJA+1" y "BAJA+2" y solo algunos "CONTINIA"

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train_final[, azar := runif(nrow(dataset_train_final))]
dataset_train_final[, training := 0L]

dataset_train_final[
  (azar <= PARAM$train_final$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

dataset_train_final[, azar:= NULL] # elimino la columna azar

### Target Engineering

In [42]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train_final[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

### Adaptacion Hiperparametros Optimos

Solamente escalo min_data_in_leaf  por  nrow(dataset_train_final) / nrow(dtrain)

In [43]:
# leo el archivo donde quedaron los hiperparametros optimos
tb_BO <-  fread("BO_log.txt")
setorder( tb_BO, -metrica)  # ordeno por metrica descendente
tb_BO[1]

fecha,objective,metric,first_metric_only,boost_from_average,feature_pre_filter,verbosity,force_row_wise,seed,extra_trees,⋯,max_bin,num_iterations,learning_rate,feature_fraction,min_data_in_leaf,num_leaves,iter,metrica_mejor,metrica_sd,metrica
<dbl>,<chr>,<chr>,<lgl>,<lgl>,<lgl>,<int>,<lgl>,<int>,<lgl>,⋯,<int>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,<dbl>,<lgl>,<dbl>
20251107,binary,custom,TRUE,TRUE,FALSE,-100,TRUE,974411,FALSE,⋯,31,2185,0.03765917,0.3817213,167,217,40,364699510,NA,364699510


In [44]:
# en la tabla ademas de los parametros del LightGBM, hay campos de salida
param_lgbm <- union( names(PARAM$lgbm$param_fijos),  names(PARAM$hipeparametertuning$hs$pars) )

PARAM$train_final$param_mejores <- as.list( tb_BO[1, param_lgbm, with=FALSE])

PARAM$train_final$param_mejores$min_data_in_leaf <- as.integer( round(PARAM$train_final$param_mejores$min_data_in_leaf * nrow(dataset_train_final[training == 1L]) / nrow(dtrain)))

cat( tb_BO[1, min_data_in_leaf] , PARAM$train_final$param_mejores$min_data_in_leaf, "\n")
PARAM$train_final$param_mejores

167 338 


$objective
[1] "binary"

$metric
[1] "custom"

$first_metric_only
[1] TRUE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$verbosity
[1] -100

$force_row_wise
[1] TRUE

$seed
[1] 974411

$extra_trees
[1] FALSE

$max_depth
[1] -1

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0

$lambda_l2
[1] 0

$bagging_fraction
[1] 1

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$max_bin
[1] 31

$num_iterations
[1] 2185

$learning_rate
[1] 0.03765917

$feature_fraction
[1] 0.3817213

$min_data_in_leaf
[1] 338

$num_leaves
[1] 217

### Final Models

Aqui SIEMPRE voy a hacer un semillerio, independientemente de si en la Bayesian Optimization calculé un semillerio en cada iteración.
<br> Entreno un LightGBM para cada semilla,  y guardo el modelo dentro de la carpeta  **modelitos**
<br> Intencionalmente en una primera etapá se generan los modelos y graban, y en una segunda etapa se leen eso modelos y se aplican a los datos del futuro

APO controla cuantas veces se repite el modelo, que se usa para promediar ganancias y reportar en la Pseudo Competencia algo razonable
<br> El modelo puede ser un LightGBM simple (ksemillerio==1)  o un Ensemble Semillerio( ksemillerio > 1 )
<br> Lamentablmente APO necesita utilizar muchas semillas, y eso demanda TIEMPO de corrida

In [45]:
# Semillerio Final
PARAM$train_final$APO <- 10
PARAM$train_final$ksemillerio  <- 10

PARAM$train_final$cortes <- c(10500, 11000, 11500, 12000)

In [46]:
# valvula de seguridad anti atajos
if( PARAM$train_final$APO * PARAM$train_final$ksemillerio  < 100
  | PARAM$train_final$APO < 5
)  {
  # stop("No quiera overfitear")
  cat("No quiera overfitear")
}

In [47]:
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
PARAM$train_final$semillas <- sample(primos)[seq( PARAM$train_final$ksemillerio*PARAM$train_final$APO )]
PARAM$train_final$semillas

[1] 974411 507673 696271 281233 677309 180647 251519 235241 176213 162601
 [11] 138637 519551 807931 471301 152063 823663 874337 466201 206191 787939
 [21] 361327 320141 459847 246833 208799 381371 868327 745187 297371 244147
 [31] 775739 486043 903163 228989 714223 766943 838157 492253 954757 698393
 [41] 424103 964637 581239 315739 868379 589439 114487 645131 794953 356567
 [51] 609821 660799 957059 495629 227627 647161 499327 671837 810239 776219
 [61] 205663 681781 766739 671633 450601 204583 822839 217411 333071 823033
 [71] 631471 595073 139663 564301 459749 713863 979171 119569 935303 440641
 [81] 207187 907811 552263 281839 990281 326707 346739 451783 523771 377129
 [91] 783613 720901 268843 349409 352619 473353 847601 859787 427241 365201

In [48]:
# dejo los datos en formato LightGBM
dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train_final[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train_final[training == 1L, clase01],
  free_raw_data= FALSE
)

cat("filas", nrow(dtrain_final), "columnas", ncol(dtrain_final), "\n")
Sys.time()

filas 437055 columnas 1241 


[1] "2025-11-07 11:17:41 EST"

In [ ]:
# genero los modelitos
dir.create( "modelitos", showWarnings= FALSE)

param_completo <- copy( PARAM$train_final$param_mejores)

for( sem in PARAM$train_final$semillas ) {

  arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
  if( !file.exists( arch_modelo ) )
  {
    param_completo$seed <- sem

    modelito <- lgb.train(
      data= dtrain_final,
      param= param_completo
    )

    lgb.save( modelito, filename= arch_modelo)
    rm(modelito)
    gc()
  }
}

Sys.time()

### Scoring

Se hace el predict() del modelo en los datos del futuro

In [ ]:
# dataset de future, donde en este caso estoy haciendo testing
dfuture <- dataset[foto_mes %in% PARAM$train_final$future ]
mfuture <- data.matrix(dfuture[, campos_buenos, with= FALSE])

dfuture[, ganancia := ifelse(clase_ternaria=="BAJA+2", 780000, -20000)]

In [ ]:
mganancias <- matrix( nrow=PARAM$train_final$APO, ncol= length(PARAM$train_final$cortes) )

if( file.exists("prediccion.txt") )
  file.remove("prediccion.txt")

In [ ]:
# aplico el modelo a los datos del future

for( vapo in seq(PARAM$train_final$APO) ) {
  # inicializacion en CERO
  vpred_acum <- rep(0.0, nrow(dfuture))
  qacumulados <- 0

  desde <- 1 + (vapo-1)*PARAM$train_final$ksemillerio
  hasta <- desde + PARAM$train_final$ksemillerio - 1
  semillas <- PARAM$train_final$semillas[desde:hasta]

  for( sem in semillas ) {

    arch_modelo <- paste0("./modelitos/mod_", sem, ".txt")
    if( file.exists( arch_modelo ) )
    {
      modelo_final <- lgb.load(arch_modelo) # leo del disco
      # hago el predict() y acumulo
      vpred_acum <- vpred_acum + predict(modelo_final, mfuture)
      qacumulados <- qacumulados + 1
      rm(modelo_final)
      gc()
    }
  }

  if( qacumulados > 0 ) {
    vpred_acum <- vpred_acum / qacumulados  # paso a probabildiad
    # tabla de prediccion, puede ser util para futuros ensembles
    #  ya que le modelo ganador va a ser un ensemble de LightGBMs

    tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes, ganancia)]
    tb_prediccion[, meta_modelo := vapo]
    tb_prediccion[, prob := vpred_acum ]
    setorder( tb_prediccion, -prob )
    tb_prediccion[, gan_acum := cumsum(ganancia)]
    tb_prediccion[, ganancia := NULL ]

    # acumulo las ganancias
    for( icor in seq(length(PARAM$train_final$cortes)) ){
      mganancias[ vapo, icor ] <- tb_prediccion[ PARAM$train_final$cortes[icor], gan_acum ]
    }

    # grabo las probabilidades del modelo
    fwrite(tb_prediccion,
      file= "prediccion.txt",
      sep= "\t",
      append= TRUE
    )

    rm(tb_prediccion)
    gc()
  }
}

Sys.time()

In [ ]:
mganancias

In [ ]:
library(ggplot2)
library(ggrepel)
library(data.table)
library(scales)

# Supongamos que tenés algo como esto:
# tb_todas = data.table(clientes, ganancia_total, semilla)
# tb_resultados_ensamble = data.table(clientes, ganancia_ensamble_real)

# Calcular máximo
maximo_punto <- tb_resultados_ensamble[ganancia_ensamble_real == max(ganancia_ensamble_real, na.rm = TRUE)]

# Crear gráfico
ggplot() +
  # Curvas de cada semilla
  geom_line(
    data = tb_todas,
    aes(x = clientes, y = ganancia_total, color = semilla),
    alpha = 0.5, linewidth = 0.6
  ) +
  # Curva del ensamble
  geom_line(
    data = tb_resultados_ensamble,
    aes(x = clientes, y = ganancia_ensamble_real),
    color = "black", linewidth = 1.2
  ) +
  # Punto máximo
  geom_point(
    data = maximo_punto,
    aes(x = clientes, y = ganancia_ensamble_real),
    color = "red", size = 3
  ) +
  geom_label_repel(
    data = maximo_punto,
    aes(
      x = clientes,
      y = ganancia_ensamble_real,
      label = paste0("Máximo\n", format(ganancia_ensamble_real, big.mark = ".", decimal.mark = ","))
    ),
    fill = "white",
    color = "red",
    fontface = "bold",
    box.padding = unit(0.4, "lines"),
    nudge_y = 5e7
  ) +
  labs(
    title = "Ganancia Acumulada (Semillas y Ensamble)",
    subtitle = "Experimento: workflowtest",
    x = "Clientes ordenados por probabilidad (índice / envíos)",
    y = "Ganancia Acumulada",
    color = "Modelo"
  ) +
  scale_y_continuous(labels = comma) +
  theme_minimal() +
  theme(
    legend.position = "right",
    legend.key.size = unit(0.5, "lines"),
    plot.title = element_text(face = "bold", size = 14)
  )
